In [1]:
from code.coref_utils.conll import evaluate_conll
from code.coref_utils.utils import mention_to_cluster
from code.coref_utils.metrics import CorefEvaluator
from os import path
import os
import glob
import json

import logging

logging.basicConfig(format='%(asctime)s - %(message)s', level=logging.INFO)


In [2]:
def load_jsonl(file_path):
    data = []
    with open(file_path, 'r') as f:
        for line in f.readlines():
            data.append(json.loads(line.strip()))
    
    return data

In [3]:
ontonotes_log_dir="/home/shtoshni/Research/litbank_coref/models/ontonotes_logs"
conll_dir = "/home/shtoshni/Research/litbank_coref/data/ontonotes/conll"
conll_scorer = "/home/shtoshni/Research/litbank_coref/resources/reference-coreference-scorers/scorer.pl"

ontonotes_files = glob.glob(path.join(ontonotes_log_dir, "*.jsonl"))
print(ontonotes_files)

['/home/shtoshni/Research/litbank_coref/models/ontonotes_logs/ontonotes_learned_10.jsonl', '/home/shtoshni/Research/litbank_coref/models/ontonotes_logs/ontonotes_lru_10.jsonl', '/home/shtoshni/Research/litbank_coref/models/ontonotes_logs/ontonotes_lru_20.jsonl', '/home/shtoshni/Research/litbank_coref/models/ontonotes_logs/ontonotes_lru_5.jsonl', '/home/shtoshni/Research/litbank_coref/models/ontonotes_logs/ontonotes_unbounded.jsonl', '/home/shtoshni/Research/litbank_coref/models/ontonotes_logs/ontonotes_learned_20.jsonl', '/home/shtoshni/Research/litbank_coref/models/ontonotes_logs/ontonotes_learned_5.jsonl']


In [4]:
def log_file_to_model(log_file):
    model_name = path.basename(log_file).split(".")[0]
#     print(model_name)
    return model_name

In [5]:
for log_file in ontonotes_files:
    model_name = log_file_to_model(log_file)
    data = load_jsonl(log_file)
    coref_predictions, subtoken_maps = {}, {}
    for example in data:
        predicted_clusters, mention_to_predicted =\
            mention_to_cluster(example["predicted_clusters"], threshold=2)
        gold_clusters, mention_to_gold =\
            mention_to_cluster(example["clusters"], threshold=2)
        
        coref_predictions[example["doc_key"]] = predicted_clusters
        subtoken_maps[example["doc_key"]] = example["subtoken_map"]
        
    split='dev'
    gold_path = path.join(conll_dir, f'{split}.conll')
    prediction_file = path.join(ontonotes_log_dir, f'{model_name}.conll')
    conll_results = evaluate_conll(
        conll_scorer, gold_path, coref_predictions, subtoken_maps, prediction_file)
    average_f1 = sum(results for results in conll_results.values()) / len(conll_results)
    logging.info("%s (CoNLL) F-score : %.1f, MUC: %.1f, Bcub: %.1f, CEAFE: %.1f"
                 % (model_name, average_f1, conll_results["muc"], conll_results['bcub'],
                    conll_results['ceafe']))

2020-06-03 09:56:07,689 - ontonotes_learned_10 (CoNLL) F-score : 76.6, MUC: 82.9, Bcub: 75.5, CEAFE: 71.4
2020-06-03 09:56:11,886 - ontonotes_lru_10 (CoNLL) F-score : 75.2, MUC: 81.9, Bcub: 73.9, CEAFE: 69.9
2020-06-03 09:56:16,038 - ontonotes_lru_20 (CoNLL) F-score : 77.5, MUC: 83.5, Bcub: 76.4, CEAFE: 72.7
2020-06-03 09:56:20,206 - ontonotes_lru_5 (CoNLL) F-score : 69.1, MUC: 77.2, Bcub: 67.4, CEAFE: 62.6
2020-06-03 09:56:24,392 - ontonotes_unbounded (CoNLL) F-score : 77.7, MUC: 83.5, Bcub: 76.6, CEAFE: 73.1
2020-06-03 09:56:28,534 - ontonotes_learned_20 (CoNLL) F-score : 77.7, MUC: 83.5, Bcub: 76.5, CEAFE: 73.0
2020-06-03 09:56:32,676 - ontonotes_learned_5 (CoNLL) F-score : 73.2, MUC: 80.3, Bcub: 71.8, CEAFE: 67.4
